<a href="https://colab.research.google.com/github/NilaknaW/llminator-app/blob/master/win-predict/llmin_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense

In [ ]:
data = {
    'month': [3, 4, 4, 5, 6, 7, 8, 8],
    'anura': [41, 40, 46, 39, 38, 37, 36, 43],
    'sajith': [39, 39, 22, 39, 38, 36, 32, 22],
    'ranil': [12, 13, 18, 15, 18, 22, 28, 27],
    'other': [8, 8, 14, 7, 6, 5, 4, 8],
    'size': [3713, 3713, 2048, 3713, 3713, 3713, 1153, 3900]
}
df = pd.DataFrame(data)
df

,month,anura,sajith,ranil,other,size
0,3,41,39,12,8,3713
1,4,40,39,13,8,3713
2,4,46,22,18,14,2048
3,5,39,39,15,7,3713
4,6,38,38,18,6,3713
5,7,37,36,22,5,3713
6,8,36,32,28,4,1153
7,8,43,22,27,8,3900


In [ ]:
# Features and targets
X = df[['month', 'size']].values
y = df[['anura', 'sajith', 'ranil']].values

In [ ]:
# Scale data
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

In [ ]:
# Reshape for LSTM (samples, timesteps, features)
X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)


In [ ]:
# Build LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(3))  # Three outputs for three candidates
model.compile(optimizer='adam', loss='mean_squared_error')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
# Train model
model.fit(X_train, y_train, epochs=20, batch_size=1, verbose=1)

Epoch 1/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4028
Epoch 2/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3889 
Epoch 3/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3805 
Epoch 4/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3844 
Epoch 5/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3642 
Epoch 6/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3360 
Epoch 7/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3316 
Epoch 8/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3306 
Epoch 9/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2996 
Epoch 10/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3097 
Epoch 11/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2983 
Epoch 12/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2946 
Epoch 13/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2716 
Epoch 14/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2660 
Epoch 15/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2510 
Epoch 16/20
6/6 ━━━━━━━━━━━━━━━━━━━

In [ ]:
# Predict
y_pred_scaled = model.predict(X_test)
y_pred = scaler_y.inverse_transform(y_pred_scaled)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step


In [ ]:
# Display predictions
print("Predicted votes for each candidate:")
print(y_pred)

Predicted votes for each candidate:
[[38.623726 26.753958 15.905848]
 [39.10293  27.813473 17.570423]]


In [ ]:
# New data point: month 9, size 17.1 million
new_data = np.array([[9, 17100000]])  # Scale size to match the input format

# Scale the new data point using the fitted scaler
new_data_scaled = scaler_X.fit_transform(new_data)

# Reshape to fit the LSTM input format (samples, timesteps, features)
new_data_scaled = new_data_scaled.reshape((new_data_scaled.shape[0], 1, new_data_scaled.shape[1]))

# Make the prediction
new_pred_scaled = model.predict(new_data_scaled)

# Inverse transform the prediction to get the original values
new_pred = scaler_y.inverse_transform(new_pred_scaled)

# Display the prediction
print("Predicted votes for month 9 with a size of 17.1 million:")
print(new_pred)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 276ms/step
Predicted votes for month 9 with a size of 17.1 million:
[[37.61226  24.388035 14.393796]]
